In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# Plot
plot_directory = 'result/plot_set'

In [3]:
# colors = [
#     '#e6194B',
#     '#f58231',
#     '#9A6324',
#     '#911eb4',
#     '#3cb44b',
#     '#f032e6',
#     '#4363d8',
# ]

In [4]:
colors = [
	'#FF0000',
	'#00FFFF',
	'#0000FF',
	'#00008B',
	'#ADD8E6',
	'#800080',
	'#7FFFD4',
	'#008000',
	'#FF00FF',
	'#FFC0CB',
	'#C0C0C0',
	'#FFA500',
	'#000000',
	'#800000',
]

In [5]:
def load_json_file(file_path):
	try:
		with open(file_path, 'r') as file:
			data = json.load(file)
		return data
	
	except Exception as e:
		print(f"An error occurred while loading the JSON file: {e}")
		return None

In [6]:
def extract_fpss(metric_list):
	return list(metric_list[list(metric_list.keys())[0]][0]['metric'].keys())

In [7]:
def to_accuracy_vector(accuracy_result_seq, fpss, type='F1'):
	accuracy_vector = []
	for fps in fpss:
		accuracy_vector.append(accuracy_result_seq[fps][type])
	
	return accuracy_vector

In [8]:
def plot_scatter(xs, ys, x_label, y_label, title, fig_size=(8, 8)):
	plt.figure(figsize=fig_size)
	plt.scatter(xs, ys, c=colors[0], s=30, alpha=0.5)
	plt.title(title)
	plt.xlabel(x_label)
	plt.ylabel(y_label)
	plt.show()

In [9]:
def plot_scatter_label(xs, ys, labels, x_label, y_label, title, fig_size=(8, 8)):
	plt.figure(figsize=fig_size)
	for i in range(len(xs)):
		plt.scatter(xs[i], ys[i], c=colors[labels[i]], s=30, alpha=0.5)
	plt.title(title)
	plt.xlabel(x_label)
	plt.ylabel(y_label)
	plt.show()

In [10]:
def filter_list_index(input_list, indices_to_remove):
	return [item for i, item in enumerate(input_list) if i not in indices_to_remove]

In [11]:
def round_float_to_sigfigs(number, sigfigs):
	return round(number, sigfigs)

In [12]:
def round_floats_to_sigfigs(input_list, sigfigs):
	return [round(number, sigfigs) for number in input_list]

In [13]:
def normalize_list(input_list):
    # Check if the list is not empty
    if not input_list:
        return []

    min_val = min(input_list)
    max_val = max(input_list)

    # Handle the case where all elements in the list are the same
    if min_val == max_val:
        return [0.5] * len(input_list)

    return [(x - min_val) / (max_val - min_val) for x in input_list]


## Plot

In [14]:
omv_features = ["Left-Top", "Right-Top", "Left-Bottom", "Right-Bottom", "Object-Amount", "Confidence", "IOU"]

In [15]:
plot_filenames = sorted(os.listdir(plot_directory))
plot_video_names = sorted(list(set([f.split('_')[0] for f in plot_filenames])))

In [16]:
fpss = extract_fpss(load_json_file(os.path.join(plot_directory, plot_video_names[0] + "_Accuracy_Result.json")))

In [17]:
omv_videos = []
acc_videos = []

for v in plot_video_names:
	omv_dict = {}
	for fps in fpss:
		omv_dict[fps] = []
	acc_list = []

	accuracy_result = load_json_file(os.path.join(plot_directory, v + "_Accuracy_Result.json"))
	movement_result = load_json_file(os.path.join(plot_directory, v + "_Movement_Result.json"))

	for class_idx in list(accuracy_result.keys()):
		for i in range(len(accuracy_result[class_idx])):
			accuracy_vector = to_accuracy_vector(accuracy_result[class_idx][i]['metric'], fpss)
			acc_list.append(accuracy_vector)

			for fps in fpss:
				movement_vector = movement_result[class_idx][i]['movement'][fps]
				omv_dict[fps].append(movement_vector)
	
	omv_videos.append(omv_dict)
	acc_videos.append(acc_list)

In [18]:
def iterate_combinations_7vars(total_sum=10):
    combinations = []
    for x1 in range(total_sum + 1):
        for x2 in range(total_sum - x1 + 1):
            for x3 in range(total_sum - x1 - x2 + 1):
                for x4 in range(total_sum - x1 - x2 - x3 + 1):
                    for x5 in range(total_sum - x1 - x2 - x3 - x4 + 1):
                        for x6 in range(total_sum - x1 - x2 - x3 - x4 - x5 + 1):
                            x7 = total_sum - x1 - x2 - x3 - x4 - x5 - x6
                            combinations.append((x1, x2, x3, x4, x5, x6, x7))
    return combinations

In [19]:
combinations = iterate_combinations_7vars(10)

In [20]:
for k in range(len(fpss)):
	fps = fpss[k]
	print(f"FPS: {fps}")

	best_average = -1.
	best_corr = None
	best_combination = None
	
	for combination in combinations:
		video_results = []
		for i in range(len(plot_video_names)):
			omv_fps_f0 = list(np.array(omv_videos[i][fps])[:, 0])
			outlier_index = [l for l in range(len(omv_fps_f0)) if omv_fps_f0[l] == -1]
			omv_fps_f0_clean = filter_list_index(omv_fps_f0, outlier_index)

			omv_weight_sum = combination[0] * np.array(normalize_list(omv_fps_f0_clean))
			for j in range(1, len(combination)):
				omv_fps = list(np.array(omv_videos[i][fps])[:, j])
				omv_fps_clean = filter_list_index(omv_fps, outlier_index)

				omv_weight_sum += combination[j] * np.array(normalize_list(omv_fps_clean))
			
			acc_fps = list(np.array(acc_videos[i])[:, k])
			acc_fps_clean = filter_list_index(acc_fps, outlier_index)
			
			correlation_matrix = np.corrcoef(omv_weight_sum, acc_fps_clean)
			correlation_coefficient = correlation_matrix[0, 1]

			video_results.append(correlation_coefficient)
		
		combination_average_corr = np.average(np.array(video_results))
		if combination_average_corr > best_average:
			best_average = combination_average_corr
			best_corr = video_results
			best_combination = combination
	print(f"Average Coefficient: {round_float_to_sigfigs(best_average, 4)}")
	print(f"Coefficients for Each Video: {round_floats_to_sigfigs(best_corr, 4)}")
	print(f"Weight: {best_combination}")
	print("")

FPS: 1
Average Coefficient: 0.5993
Coefficients for Each Video: [0.6458, 0.5082, 0.6463, 0.5968]
Weight: (0, 0, 0, 0, 5, 2, 3)

FPS: 2
Average Coefficient: 0.5693
Coefficients for Each Video: [0.6487, 0.4686, 0.6196, 0.5403]
Weight: (0, 0, 0, 0, 5, 3, 2)

FPS: 3
Average Coefficient: 0.5611
Coefficients for Each Video: [0.6307, 0.4901, 0.5962, 0.5276]
Weight: (0, 0, 0, 0, 4, 4, 2)

FPS: 5
Average Coefficient: 0.552
Coefficients for Each Video: [0.6226, 0.442, 0.5542, 0.5892]
Weight: (0, 0, 0, 0, 3, 5, 2)

FPS: 6
Average Coefficient: 0.5634
Coefficients for Each Video: [0.6782, 0.4722, 0.5187, 0.5844]
Weight: (0, 0, 0, 0, 2, 5, 3)

FPS: 10
Average Coefficient: 0.5237
Coefficients for Each Video: [0.6315, 0.4519, 0.4569, 0.5545]
Weight: (1, 0, 0, 0, 2, 5, 2)

FPS: 15
Average Coefficient: 0.5027
Coefficients for Each Video: [0.6365, 0.4299, 0.3708, 0.5737]
Weight: (1, 0, 0, 0, 2, 6, 1)

FPS: 30
Average Coefficient: 0.511
Coefficients for Each Video: [0.6876, 0.4393, 0.3299, 0.5871]
Weight:

In [28]:
def iterate_all_4vars(ss=5):
    combinations = []
    for x1 in range(-ss, ss+1):
        for x2 in range(-ss, ss+1):
            for x3 in range(-ss, ss+1):
                for x4 in range(-ss, ss+1):
                            combinations.append((x1, x2, x3, x4))
    return combinations

In [29]:
combinations_v4 = iterate_all_4vars(5)
skipped_features = 3

In [30]:
for k in range(len(fpss)):
	fps = fpss[k]
	print(f"FPS: {fps}")

	best_average = -1.
	best_corr = None
	best_combination = None
	
	for combination in combinations_v4:
		video_results = []
		for i in range(len(plot_video_names)):
			omv_fps_f0 = list(np.array(omv_videos[i][fps])[:, 0])
			outlier_index = [l for l in range(len(omv_fps_f0)) if omv_fps_f0[l] == -1]
			omv_fps_f0_clean = filter_list_index(omv_fps_f0, outlier_index)

			omv_weight_sum = combination[0] * np.array(normalize_list(omv_fps_f0_clean))
			for j in range(1, len(combination)):
				omv_fps = list(np.array(omv_videos[i][fps])[:, j + skipped_features])
				omv_fps_clean = filter_list_index(omv_fps, outlier_index)

				omv_weight_sum += combination[j] * np.array(normalize_list(omv_fps_clean))
			
			acc_fps = list(np.array(acc_videos[i])[:, k])
			acc_fps_clean = filter_list_index(acc_fps, outlier_index)
			
			correlation_matrix = np.corrcoef(omv_weight_sum, acc_fps_clean)
			correlation_coefficient = correlation_matrix[0, 1]

			video_results.append(correlation_coefficient)
		
		combination_average_corr = np.average(np.array(video_results))
		if combination_average_corr > best_average:
			best_average = combination_average_corr
			best_corr = video_results
			best_combination = combination
	print(f"Average Coefficient: {round_float_to_sigfigs(best_average, 4)}")
	print(f"Coefficients for Each Video: {round_floats_to_sigfigs(best_corr, 4)}")
	print(f"Weight: {best_combination}")
	print("")

FPS: 1


/Users/jiaxili/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2829: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/jiaxili/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2830: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Average Coefficient: 0.6148
Coefficients for Each Video: [0.6589, 0.5072, 0.6545, 0.6386]
Weight: (-1, 4, 2, 3)

FPS: 2
Average Coefficient: 0.5795
Coefficients for Each Video: [0.6604, 0.4613, 0.6324, 0.5638]
Weight: (-1, 5, 3, 3)

FPS: 3
Average Coefficient: 0.5848
Coefficients for Each Video: [0.6224, 0.498, 0.6112, 0.6075]
Weight: (-2, 5, 5, 2)

FPS: 5
Average Coefficient: 0.5546
Coefficients for Each Video: [0.6308, 0.4448, 0.5704, 0.5724]
Weight: (0, 3, 5, 3)

FPS: 6
Average Coefficient: 0.5688
Coefficients for Each Video: [0.6612, 0.4897, 0.5355, 0.5888]
Weight: (0, 1, 2, 1)

FPS: 10
Average Coefficient: 0.5237
Coefficients for Each Video: [0.6315, 0.4519, 0.4569, 0.5545]
Weight: (1, 2, 5, 2)

FPS: 15
Average Coefficient: 0.5058
Coefficients for Each Video: [0.6771, 0.419, 0.34, 0.5871]
Weight: (1, 1, 5, 1)

FPS: 30
Average Coefficient: 0.5096
Coefficients for Each Video: [0.6841, 0.4352, 0.3454, 0.5735]
Weight: (1, 1, 5, 1)

